In [1]:
import pandas as pd

In [ ]:
# df = pd.read_csv('data/data.csv', usecols=['Content', 'Summary'])

In [ ]:
# df.head()

In [ ]:
# df['Content'] = df['Content'].str.replace('\xa0', ' ', regex=False)

In [ ]:
# df = df[~df['Content'].isna()]

In [ ]:
!pip install pytesseract transformers datasets rouge-score nltk tensorboard py7zr --upgrade
# install git-fls for pushing model and logs to the hugging face hub
!sudo apt-get install git-lfs --yes

In [45]:
df = pd.read_csv('data/data.csv', usecols=['Content', 'Summary'])

In [46]:
df.to_csv('data/data.csv', index=False)

In [47]:
df.head()

,Content,Summary
0,New York police are concerned drones could bec...,Police have investigated criminals who have ri...
1,By . Ryan Lipman . Perhaps Australian porn sta...,Porn star Angela White secretly filmed sex act...
2,"This was, Sergio Garcia conceded, much like be...",American draws inspiration from fellow country...
3,An Ebola outbreak that began in Guinea four mo...,World Health Organisation: 635 infections and ...
4,By . Associated Press and Daily Mail Reporter ...,A sinkhole opened up at 5:15am this morning in...


In [34]:
import datasets
# ri = datasets.ReadInstruction("train") + datasets.ReadInstruction("test")
df = load_dataset('csv', data_files='data/data.csv', split='train')

Using custom data configuration default-6ffdf81960e13284
Found cached dataset csv (/Users/han/.cache/huggingface/datasets/csv/default-6ffdf81960e13284/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [39]:
dataset = df.train_test_split(test_size=.2)

In [40]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ID', 'Content', 'Summary', 'Dataset'],
        num_rows: 696416
    })
    test: Dataset({
        features: ['Unnamed: 0', 'ID', 'Content', 'Summary', 'Dataset'],
        num_rows: 174105
    })
})

In [41]:
from datasets import load_dataset

# dataset_id = "samsum"

# Load dataset from the hub
# dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Train dataset size: 696416
Test dataset size: 174105


In [42]:
from random import randrange


sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"dialogue: \n{sample['Content']}\n---------------")
print(f"summary: \n{sample['Summary']}\n---------------")

dialogue: 
Captain Amla finally departed for 201, while Bavuma made an unbeaten 102 as the Proteas declared on 627-7, two short of England's first-innings total.
Faf du Plessis hit 86, while Chris Morris contributed 69 after the hosts lost three wickets for 10 runs.
England reached 16-0 at the close, leading by 18.
Openers Alastair Cook and Alex Hales successfully negotiated a testing six overs in the final half hour.
A draw seems the most likely outcome on a flat Newlands wicket but South Africa will be looking to dismiss the tourists cheaply on day five to set up an unlikely triumph and square the four-match series.
In reaching 100 off 141 balls, Bavuma also became the first black African to score a Test century for South Africa as the hosts' middle order overcame their struggles in the first Test to make England toil.
England squandered numerous chances to exert control and potentially set up victory by dropping eight catches in South Africa's innings, which contributed to them spen

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-base"

# Load tokenizer of FLAN-t5-base
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [44]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'ID', 'Content', 'Summary', 'Dataset'],
        num_rows: 696416
    })
    test: Dataset({
        features: ['Unnamed: 0', 'ID', 'Content', 'Summary', 'Dataset'],
        num_rows: 174105
    })
})

In [43]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["Content"], truncation=True), batched=True, remove_columns=["Content", "Summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["Summary"], truncation=True), batched=True, remove_columns=["Content", "Summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

  3%|██                                                                        | 24/871 [00:14<08:20,  1.69ba/s]


TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [8]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")


Loading cached processed dataset at /Users/han/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e/cache-8b1a346b93a0e067.arrow
100%|█████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.29ba/s]
Loading cached processed dataset at /Users/han/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e/cache-8c7b42f0773d37d9.arrow


Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [9]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result


[nltk_data] Downloading package punkt to /Users/han/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)


Downloading: 100%|██████████████████████████████████████████████████████████| 1.40k/1.40k [00:00<00:00, 275kB/s]
Downloading: 100%|███████████████████████████████████████████████████████████| 990M/990M [02:00<00:00, 8.21MB/s]


In [11]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)



In [12]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False,
    hub_strategy="every_save",
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)


In [13]:
# Start training
trainer.train()


/Users/han/Library/Python/3.9/lib/python/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 14732
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9210
  Number of trainable parameters = 247577856


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()


In [ ]:
df['Content'][1]

In [ ]:
df['Content'][0]

In [ ]:
df['Summary'][0]